In [1]:
import pandas as pd
import gc

In [2]:
train = pd.read_feather('Feathers/ss_alltrain_typechanged.ftr')
print('loaded train')
test = pd.read_feather('Feathers/ss_test_via_alltrain_typechanged.ftr')
print('loaded test')

loaded train
loaded test


In [4]:
train.set_index('index', inplace=True)
del train.index.name
test.set_index('index', inplace=True)
del test.index.name

In [5]:
train.head(2)

,app,app_day_hour_count,app_device_os_mean,attributed_timediff,channel,device,hour,ip_app_day_hour_count,ip_app_device_countfromfuture,ip_app_device_countfrompast,...,ip_device_os_lasttimediff,ip_device_os_mean,ip_device_os_time2nextclick,ip_device_os_time2previousclick,ip_os_day_hour_count,is_attributed,matrixFact_user_ip_item_appdeviceos,matrixFact_user_ipchannel_item_appdeviceos,matrixFact_user_iposdeviceapp_item_app,os
0,3,-1.355859,-0.083193,0.482835,133,1,14,-0.231806,0.354194,-0.21365,...,2.718168,0.027336,1.292117,-0.207378,-0.221148,0.0,-0.269234,-0.426699,-1.119404,13
1,3,-1.355859,-0.084415,0.482835,133,1,14,-0.231806,0.262257,-0.21365,...,2.711555,0.086107,1.272907,-0.207378,-0.221148,0.0,-0.984914,-0.612455,-0.528700,19


In [6]:
test.head(2)

,app,app_day_hour_count,app_device_os_mean,attributed_timediff,channel,device,hour,ip_app_day_hour_count,ip_app_device_countfromfuture,ip_app_device_countfrompast,...,ip_device_os_lasttimediff,ip_device_os_mean,ip_device_os_time2nextclick,ip_device_os_time2previousclick,ip_os_day_hour_count,is_attributed,matrixFact_user_ip_item_appdeviceos,matrixFact_user_ipchannel_item_appdeviceos,matrixFact_user_iposdeviceapp_item_app,os
0,9,0.790197,-0.0755,0.091585,17,1,4,-0.215741,-0.197744,-0.192336,...,-1.132750,0.008702,-0.205401,-0.205966,-0.217264,NaN,0.521780,0.467860,1.140992,3
1,9,0.790197,-0.0755,-0.343579,177,1,4,-0.119355,-0.060634,-0.101353,...,0.553337,0.201479,-0.204271,-0.205401,-0.190076,NaN,0.239275,0.255219,0.778947,3


ori_path = '/home/kai/data/kaggle/talkingdata/data/'

test_ori = pd.read_csv(ori_path+"test.csv", dtype=dtypes, usecols=['ip','app','device','os', 'channel', 'click_time', 'click_id'])

In [7]:
embedding_features = ['app','channel','device','os','hour']

numeric_features = list(set(set(train.columns) - set(embedding_features)) - set(['is_attributed']))
print(numeric_features, len(numeric_features))

['app_day_hour_count', 'ip_app_device_time2nextclick', 'matrixFact_user_ip_item_appdeviceos', 'ip_app_day_hour_count', 'ip_device_os_time2nextclick', 'attributed_timediff', 'app_device_os_mean', 'ip_app_device_os_lasttimediff', 'ip_app_device_firsttimediff', 'ip_app_device_os_countfromfuture', 'ip_app_device_os_countfrompast', 'ip_app_device_os_time2previousclick', 'matrixFact_user_ipchannel_item_appdeviceos', 'ip_app_os_day_hour_count', 'ip_device_os_countfrompast', 'ip_app_device_countfromfuture', 'ip_device_os_time2previousclick', 'ip_app_device_mean', 'ip_device_os_firsttimediff', 'ip_app_device_os_firsttimediff', 'ip_app_device_countfrompast', 'ip_app_device_os_mean', 'ip_device_os_mean', 'ip_app_device_os_time2nextclick', 'ip_os_day_hour_count', 'ip_app_device_time2previousclick', 'ip_device_os_count', 'ip_device_os_lasttimediff', 'ip_day_hour_count', 'matrixFact_user_iposdeviceapp_item_app', 'ip_device_os_countfromfuture', 'ip_app_device_os_count', 'ip_app_device_lasttimediff'] 

In [12]:
def get_keras_data(dataset):
    X = dict((col, np.array(dataset[col])) for col in embedding_features)
    X_num =  dict(('num_'+col, np.array(dataset[col])) for col in numeric_features)
    X.update(X_num)
    return X

In [8]:
len(numeric_features)

33

In [9]:
train.shape, test.shape

((184903890, 39), (18790469, 39))

In [10]:
train['ip_device_os_lasttimediff'].std(), test['ip_device_os_lasttimediff'].std()

(1.0, 0.6089447)

In [11]:
from keras.models import Model
from keras.layers import Dense, Input, Embedding, Dropout, Activation, Reshape, Flatten
from keras.layers.merge import concatenate, dot, add, multiply
from keras.layers.normalization import BatchNormalization
from keras.layers import SpatialDropout1D, Conv1D
from keras.layers.advanced_activations import LeakyReLU, PReLU, ELU
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from keras.regularizers import l1, l2, l1_l2
from keras.initializers import RandomUniform
from keras.optimizers import RMSprop, Adam, SGD

import numpy as np

/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [13]:
import gc; gc.collect()

176

In [14]:
df_hour = pd.read_csv('/home/kai/data/kaggle/talkingdata/data/hourdistri.csv', index_col='Unnamed: 0')
index = {}
for day in ['day7']:#, 'day8','day9']:
    index[day] = list(range(df_hour.loc[day,'4start'], df_hour.loc[day,'6end0sec'])) + \
    list(range(df_hour.loc[day,'9start'], df_hour.loc[day,'11end0sec'])) + \
    list(range(df_hour.loc[day,'13start'], df_hour.loc[day,'15end0sec'])) 

In [22]:
forval = train.index.isin(index['day7'])

assert np.sum(forval) == 19534560
assert len(forval) == 184903890

day8start = int(len(train)*1/3)
day8start

day9start = int(len(train)*2/3)
day9start

del train_df, y_train, val_df, y_val; gc.collect()

In [208]:
import sys

sys.getsizeof(train)/1024**3

32.208416391164064

In [202]:
np.iinfo(np.int16).min, np.iinfo(np.int16).max

(-32768, 32767)

In [28]:
train_df = get_keras_data(train[~forval])#[day8start:])#train.iloc[index['day8']+index['day9']])#:(day9len+day8len)])
y_train = train[~forval]['is_attributed'].values

len(y_train), len(train_df['app'])

(165369330, 165369330)

In [27]:
val_df = get_keras_data(train[forval])#[(day9len+day8len):(day9len+day8len+day7len)])
y_val = train[forval]['is_attributed'].values

len(y_val), len(val_df['app'])

(19534560, 19534560)

In [222]:
test_df = get_keras_data(test)

len(test_df), len(test_df['app'])

In [29]:
from sklearn.metrics import roc_auc_score
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1, batch_size=50000):
        super(Callback, self).__init__()
        print('RocAuc evaluating batch size is: {}'.format(batch_size))
        self.interval = interval
        self.X_val, self.y_val = validation_data
        self.batch_size = batch_size

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            print('\non epoch end, start predicting validation set...')
            y_pred = self.model.predict(self.X_val, batch_size=self.batch_size,verbose=1)
            print('start calculating ROC-AUC...')
            score = roc_auc_score(self.y_val, y_pred)
            print("ROC-AUC - epoch: {:d} - score: {:.6f}".format(epoch+1, score))
            
            
import keras
class RocAucMetricCallback(keras.callbacks.Callback):
    def __init__(self, validation_data=(), predict_batch_size=100000, include_on_batch=False):
        super(RocAucMetricCallback, self).__init__()
        self.predict_batch_size=predict_batch_size
        self.include_on_batch=include_on_batch
        self.X_val, self.y_val = validation_data

    def on_batch_begin(self, batch, logs={}):
        pass

    def on_batch_end(self, batch, logs={}):
        if(self.include_on_batch):
            logs['roc_auc_val']=float('-inf')
            if(self.validation_data):
                logs['roc_auc_val']=roc_auc_score(self.y_val,
                                              self.model.predict(self.X_val,
                                                                 batch_size=self.predict_batch_size))
                
    def on_train_begin(self, logs={}):
        if not ('roc_auc_val' in self.params['metrics']):
            self.params['metrics'].append('roc_auc_val')

    def on_train_end(self, logs={}):
        pass

    def on_epoch_begin(self, epoch, logs={}):
        pass

    def on_epoch_end(self, epoch, logs={}):
        logs['roc_auc_val']=float('-inf')
#         y_val = self.validation_data[-3]
#         val_df = self.validation_data[:-3]
        if(self.validation_data):
            logs['roc_auc_val']=roc_auc_score(self.y_val,
                                              self.model.predict(self.X_val,
                                                                 batch_size=self.predict_batch_size))
#             logs['roc_auc_val']=roc_auc_score(y_val,#self.validation_data[1], 
#                                               self.model.predict(val_df,#self.validation_data[0],
#                                                                  batch_size=self.predict_batch_size))

In [31]:
max_dict = dict(train[embedding_features].max())
max_dict

{'app': 729, 'channel': 201, 'device': 3798, 'hour': 23, 'os': 855}

In [ ]:
#embids = embedding_features#['app', 'channel', 'device', 'os', 'hour', 'day', 'wday', 'qty', 'ip_app_count', 'ip_app_os_count']
# get the max of each code type
embmaxs = dict((col, max_dict[col]+1) for col in embedding_features)

emb_n = 50

# Build the inputs, embeddings and concatenate them all for each column
emb_inputs = dict((col, Input(shape=[1], name = col))  for col in embedding_features)
emb_model  = dict((col, Embedding(embmaxs[col], emb_n)(emb_inputs[col])) for col in embedding_features)

fe = concatenate([(emb_) for emb_ in emb_model.values()])

s_dout = SpatialDropout1D(0.2)(fe)

conv = Conv1D(100, kernel_size=4, strides=1, padding='same')(s_dout)

num_inputs = dict((col, Input(shape=[1], name = 'num_'+col))  for col in numeric_features)

num_fe = concatenate([(num_) for num_ in num_inputs.values()])

concat = concatenate([Flatten()(s_dout), Flatten()(conv), (num_fe)])

dense_n = 1000

x = Dropout(0.2)(Dense(dense_n,activation='relu')(concat))
x = Dropout(0.2)(Dense(dense_n,activation='relu')(x))

outp = Dense(1,activation='sigmoid')(x)

model = Model(inputs=[inp for inp in emb_inputs.values()] + [inp for inp in num_inputs.values()], outputs=outp)
#model = Model(inputs=[inp for inp in emb_inputs.values()], outputs=outp)

batch_size = 200000
epochs = 4
exp_decay = lambda init, fin, steps: (init/fin)**(1/(steps-1)) - 1
steps = int(len(list(train_df)[0]) / batch_size) * epochs
lr_init, lr_fin = 0.002, 0.0002
lr_decay = exp_decay(lr_init, lr_fin, steps)
optimizer_adam = Adam(lr=0.002, decay=lr_decay)
model.compile(loss='binary_crossentropy',optimizer=optimizer_adam,metrics=['accuracy'])


# checkpoint = ModelCheckpoint(model_save_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
# earlystopping = EarlyStopping(monitor='val_loss', mode="min", patience=8)
# rocauc = RocAucEvaluation(validation_data=(val_df, y_val), interval=1, batch_size=pred_batch_size)

import time
tempid = str(int(time.time()))[3:]
print('Model ID: ' + tempid) 
seed = 7
np.random.seed(seed)
pred_batch_size = 100000
model_save_path = 'NN_MODELS/NN_data{}_batchsize{}_ep{}_{}.hdf5'.format(len(y_train), batch_size, epochs, tempid)

cb = [
    RocAucMetricCallback(validation_data=(val_df, y_val)), # include it before EarlyStopping!
    EarlyStopping(monitor='roc_auc_val',mode='max',patience=4, verbose=2),
    ModelCheckpoint(model_save_path, monitor='roc_auc_val', verbose=1, save_best_only=True, mode='max'),
#     RocAucEvaluation(validation_data=(val_df, y_val), interval=1, batch_size=pred_batch_size)
]

class_weight = {0:1,1:99}

model.fit(train_df, y_train, batch_size=batch_size, epochs=epochs, 
          validation_data=(val_df, y_val), 
#           validation_split = 0.5,
          class_weight=class_weight,
          shuffle=True, verbose=1, callbacks=cb)#[rocauc, earlystopping, checkpoint])

Model ID: 5385687
Train on 165369330 samples, validate on 19534560 samples
Epoch 1/4
134600000/165369330 [=======================>......] - ETA: 2:17 - loss: 0.1523 - acc: 0.9854

In [227]:
model.load_weights('NN_batchsize200000_ep8_5308912.hdf5')

In [228]:
val_pred_savedmodel = model.predict(val_df, batch_size=pred_batch_size, verbose=1)

19534560/19534560 [==============================] - 42s 2us/step


In [203]:
roc_auc_score(y_val, val_pred_fullmodel)

0.9817132065702936

In [230]:
roc_auc_score(y_val, val_pred_savedmodel)

0.9818143556926712

In [231]:
sub = pd.DataFrame()
sub['click_id'] = test_ori['click_id'].astype('int')

print("predicting....")
sub['is_attributed'] = model.predict(test_df, batch_size=pred_batch_size, verbose=1)

flag = 'saved'
sub.to_csv('nn_{}.csv.gz'.format(flag, tempid), index=False, float_format='%.9f', compression='gzip')

predicting....
18790469/18790469 [==============================] - 41s 2us/step


In [ ]:
# model predict val
# model predict test

# THEN load saved model 
# model predict val
# model predict test

In [66]:
roc_auc_score(y_val_1, val_pred_1) # 0.01

0.9788076360576548

In [55]:
roc_auc_score(y_val_1, val_pred_1) # 0.001

0.9761119662504989

In [39]:
embedding_inputs = []
embedding_outputs = []

In [43]:
output_dim = 50
for i in range(len(embedding_features)):
    tmp_input = Input(shape=(1,), dtype='int32', name=embedding_features[i]+'_input')
    tmp_embeddings = Embedding(int(train_embeddings[i].max()+1),output_dim,name=embedding_features[i]+'_embeddings')(tmp_input)
    tmp_embeddings = Flatten(name=embedding_features[i]+'_flatten')(tmp_embeddings)
    
    embedding_inputs.append(tmp_input)
    embedding_outputs.append(tmp_embeddings)

In [53]:
embedding_outputs.append(number_input)

In [57]:
profile = concatenate(embedding_outputs, name='profile')

In [58]:
profile

<tf.Tensor 'profile/concat:0' shape=(?, ?) dtype=float32>

In [59]:
def FunctionalDense(n, x, batchnorm=False, act='relu', lw1=0.0, dropout=0, name=''):
    if lw1 == 0.0:
        x = Dense(n, name=name+'_dense')(x)
    else:
        x = Dense(n, kernel_regularizer=l1(lw1), name=name+'_dense')(x)
    
    if batchnorm:
        x = BatchNormalization(name=name+'_batchnorm')(x)
        
    if act in {'relu', 'tanh', 'sigmoid'}:
        x = Activation(act, name=name+'_activation')(x)
    elif act =='prelu':
        x = PReLU(name=name+'_activation')(x)
    elif act == 'leakyrelu':
        x = LeakyReLU(name=name+'_activation')(x)
    elif act == 'elu':
        x = ELU(name=name+'_activation')(x)
    
    if dropout > 0:
        x = Dropout(dropout, name=name+'_dropout')(x)
        
    return x

In [62]:
profile_embedddings = FunctionalDense(100, profile, batchnorm=True, dropout=0.2, name='profile')